In [1]:
from info_nas.datasets.networks.pretrained import pretrain_network_cifar
from info_nas.datasets.networks.utils import load_nasbench
from nasbench_pytorch.datasets.cifar10 import prepare_dataset

In [2]:
seed = 42
batch_size = 32
num_workers = 8

dataset = prepare_dataset(batch_size, root='../data/', validation_size=1000, random_state=seed,
                          num_workers=num_workers)
train, n_train, val, n_val, test, n_test = dataset

--- Preparing CIFAR10 Data ---
Files already downloaded and verified
Files already downloaded and verified
--- CIFAR10 Data Prepared ---


In [3]:
print('train: ', n_train)
print('validation: ', n_val)
print('test: ', n_test)

train:  49000
validation:  1000
test:  10000


In [4]:
nasbench_path = '/home/gabi/diplomka/nasbench/nasbench_only108.tfrecord'

nb = load_nasbench(nasbench_path)

Loading dataset from file... This may take a few minutes...
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Loaded dataset in 33 seconds


In [5]:
nets = nb[:50]

In [6]:
nets[0]

('00005c142e6f48ac74fdcf73e3439874',
 ['input',
  'conv3x3-bn-relu',
  'maxpool3x3',
  'conv3x3-bn-relu',
  'conv3x3-bn-relu',
  'conv1x1-bn-relu',
  'output'],
 array([[0, 1, 0, 0, 1, 1, 0],
        [0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 1],
        [0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0]], dtype=int8))

In [7]:
from nasbench_pytorch.model import Network

num_labels = 10

def to_model(ops, mat):
    return Network((mat, ops), num_labels)

net_dict = {v[0]: to_model(v[1], v[2]) for v in nets}

In [ ]:
import torch
from info_nas.datasets.networks.utils import save_trained_net
from datetime import datetime

torch.backends.cudnn.benchmark = True
device = torch.device('cuda')

for net_hash, net in net_dict.items():
    now = datetime.now()
    print(now.strftime("%d/%m/%Y %H:%M:%S\n--------------------"))
    
    # train net and save it
    net = net.to(device)
    net, metrics = pretrain_network_cifar(net, train, val, test,
                                          num_tests=n_test, num_epochs=10, device=device)
    
    save_trained_net(net_hash, net, info=metrics, net_args=[num_labels])

24/05/2021 21:35:21
--------------------
Epoch=0/10 Batch=1/1532 | Loss=2.277, Acc=0.219(7/32)
Epoch=0/10 Batch=101/1532 | Loss=2.266, Acc=0.182(588/3232)
Epoch=0/10 Batch=201/1532 | Loss=2.132, Acc=0.225(1447/6432)
Epoch=0/10 Batch=301/1532 | Loss=2.026, Acc=0.259(2491/9632)
Epoch=0/10 Batch=401/1532 | Loss=1.933, Acc=0.289(3708/12832)
Epoch=0/10 Batch=501/1532 | Loss=1.873, Acc=0.311(4978/16032)
Epoch=0/10 Batch=601/1532 | Loss=1.820, Acc=0.330(6355/19232)
Epoch=0/10 Batch=701/1532 | Loss=1.773, Acc=0.349(7835/22432)
Epoch=0/10 Batch=801/1532 | Loss=1.732, Acc=0.365(9363/25632)
Epoch=0/10 Batch=901/1532 | Loss=1.696, Acc=0.380(10955/28832)
Epoch=0/10 Batch=1001/1532 | Loss=1.660, Acc=0.393(12595/32032)
Epoch=0/10 Batch=1101/1532 | Loss=1.631, Acc=0.406(14297/35232)
Epoch=0/10 Batch=1201/1532 | Loss=1.599, Acc=0.418(16052/38432)
Epoch=0/10 Batch=1301/1532 | Loss=1.574, Acc=0.428(17808/41632)
Epoch=0/10 Batch=1401/1532 | Loss=1.549, Acc=0.437(19602/44832)


In [ ]:
#jen 10 epochs for testing (108 trvá)
#TODO později jako v paperu (např. RMSProp)